## Perps V3 Tutorial

This tutorial will walk you through the basics of using the Synthetix python SDK to interact with Perps V3. It will cover the following:
* Connecting to the Synthetix contracts

 

In [2]:
import os
import time
from synthetix import Synthetix
from dotenv import load_dotenv

load_dotenv()

True

In [6]:
snx = Synthetix(
    provider_rpc=os.getenv('TESTNET_RPC'), # An OP Goerli RPC endpoint
    address=os.getenv('ADDRESS'), # A wallet address
    private_key=os.getenv('PRIVATE_KEY'), # A wallet private key
    network_id=420 # OP Goerli
)

In [9]:
# print the available contract names
snx.contracts.keys()


dict_keys(['PerpsV2MarketData', 'USDProxy', 'sUSD', 'PerpsMarketProxy', 'PerpsV2Market', 'AccountProxy'])

In [16]:
# fetch balances of sUSD and legacy sUSD
susd_balance = snx.get_susd_balance()
susd_legacy_balance = snx.get_susd_balance(legacy=True)

print(f"{susd_balance['balance_usd']} sUSD")
print(f"{susd_legacy_balance['balance_usd']} sUSD (legacy)")


992.568402305990172293 sUSD
151318.035943015179225061 sUSD (legacy)


In [17]:
# get a market summary
market = snx.perps.get_market_summary(100)
print(market)

{'skew': 1543200000000000000, 'size': 1654000000000000000, 'max_open_interest': 10000000000000000000000, 'current_funding_rate': -379480679791752, 'current_funding_velocity': 138888000000000, 'index_price': 1658640500000000000000}


In [20]:
# show your default account
print(f'Default account: {snx.perps.default_account_id}')

# get balances of collateral in your default account
balances = snx.perps.get_collateral_balances()
print(f'Balances: {balances}')

Default account: 69
Balances: {'sUSD': 139833958720252274652, 'BTC': 0, 'ETH': 0}


In [8]:
position = snx.perps.get_open_position(100)
print(position)

{'pnl': -71904026736662440131, 'accrued_funding': 700279455189426066, 'position_size': 900000000000000000}


In [9]:
order = snx.perps.commit_order(-0.5, market_name='ETH')
print(order)

{'from': '0x3239a95A9262034ca28b9a03133775f716f119f8', 'to': '0xf272382cB3BE898A8CdB1A23BE056fA2Fcf4513b', 'chainId': 420, 'value': 0, 'gasPrice': 1000000000, 'nonce': 2137, 'data': '0x9f97886000000000000000000000000000000000000000000000000000000000000000640000000000000000000000000000000000000000000000000000000000000045fffffffffffffffffffffffffffffffffffffffffffffffff90fa4a62c4e0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000005c80fefd54218c872d54524f59000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'}


In [9]:
snx.perps.get_order()

{'settlement_time': 1692295761,
 'market_id': 100,
 'account_id': 69,
 'size_delta': 0,
 'settlement_strategy_id': 0,
 'acceptable_price': 1776044400000000030930,
 'tracking_code': b'TROY\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00',
 'referrer': '0x0000000000000000000000000000000000000000',
 'settlement_strategy': {'strategy_type': 0,
  'settlement_delay': 15,
  'settlement_window_duration': 600,
  'price_window_duration': 180,
  'price_verification_contract': '0xff1a0f4744e8582DF1aE09D5611b887B6a12925C',
  'feed_id': b'\xca\x80\xbam\xc3.\x08\xd0o\x1a\xa8\x86\x01\x1e\xed\x1dw\xc7{\xe9\xebv\x1c\xc1\rr\xb7\xd0\xa2\xfdW\xa6',
  'url': 'https://api.synthetix.io/pyth-testnet/api/get_vaa_ccip?data={data}',
  'settlement_reward': 0,
  'price_deviation_tolerance': 50000000000000000,
  'disabled': False}}

In [12]:
# commit order
commit = snx.perps.commit_order(0.1, market_name='ETH', account_id=69, execute_now=True)


2023-08-17 12:09:04,454 - Synthetix - INFO - Committing order size 100000000000000000 to ETH (100) for 0x3239a95A9262034ca28b9a03133775f716f119f8
2023-08-17 12:09:04,456 - Synthetix - INFO - commit_order tx: 0xed1091e6572f199943c7adc167058679f53cd1bde8749cb521e45676c8715364


In [15]:
order = snx.perps.get_order()
print(f"{order['settlement_time'] - time.time()} until settlement")

11.9030020236969 until settlement


In [16]:
# settle order
settle = snx.perps.settle_pyth_order(execute_now=True)


2023-08-17 12:09:11,026 - Synthetix - INFO - settlement time: 1692295761
2023-08-17 12:09:11,028 - Synthetix - INFO - current time: 1692295751.028453
2023-08-17 12:09:11,028 - Synthetix - INFO - Waiting 9.97104787826538 seconds until order can be settled
2023-08-17 12:09:23,553 - Synthetix - INFO - Price update data not available, waiting 2 seconds and retrying
2023-08-17 12:09:26,799 - Synthetix - INFO - Price update data not available, waiting 2 seconds and retrying
2023-08-17 12:09:29,924 - Synthetix - INFO - Price update data not available, waiting 2 seconds and retrying
2023-08-17 12:09:32,336 - Synthetix - INFO - price_update_data: 0x010000000001008ef14e899127e4cef01972f14ee1bcba00bbaed0721f0b9638bc089cf3280df95f5d9f50f5c5bbfd59817606c4300e8eb332234f2295833c7ab9816135a5fca30164de625600000000001aa27839d641b07743c0cb5f68c51f8cd31d2c0762bec00dc6fcd25433ef1ab5b6000000000c9b006d0150325748000300010001020005009d431cc2fd0ef4af4bc7c85fffae2f63d51b26d162179682d149ae619b1221c00bfc309467defa